In [2]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4")

In [3]:

from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore


class RetrieverEvent(Event):
    """Result of running retrieval"""

    nodes: list[NodeWithScore]


class CreateCitationsEvent(Event):
    """Add citations to the nodes."""

    nodes: list[NodeWithScore]

from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)


In [4]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.embeddings.openai import OpenAIEmbedding

def load_or_create_index(directory_path, persist_dir):
        if os.path.exists(persist_dir):
            print("Loading existing index...")
            storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print("Creating new index...")
            documents = SimpleDirectoryReader(directory_path, recursive=True).load_data()
            index = VectorStoreIndex.from_documents(
                documents=documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
            )
            index.storage_context.persist(persist_dir=persist_dir)
        return index

index = load_or_create_index(
            "data/get-started/",
            "storage"
        )
query_engine = index.as_query_engine()

Loading existing index...


In [5]:
print(index)

In [6]:
from llama_index.core.prompts import PromptTemplate

CITATION_QA_TEMPLATE = PromptTemplate(
    "Please provide an answer based solely on the provided sources. "
    "When referencing information from a source, "
    "cite the appropriate source(s) using their corresponding numbers. "
    "Every answer should include at least one source citation. "
    "Only cite a source when you are explicitly referencing it. "
    "If none of the sources are helpful, you should indicate that. "
    "For example:\n"
    "Source 1:\n"
    "The sky is red in the evening and blue in the morning.\n"
    "Source 2:\n"
    "Water is wet when the sky is red.\n"
    "Query: When is water wet?\n"
    "Answer: Water will be wet when the sky is red [2], "
    "which occurs in the evening [1].\n"
    "Now it's your turn. Below are several numbered sources of information:"
    "\n------\n"
    "{context_str}"
    "\n------\n"
    "Query: {query_str}\n"
    "Answer: "
)

CITATION_REFINE_TEMPLATE = PromptTemplate(
    "Please provide an answer based solely on the provided sources. "
    "When referencing information from a source, "
    "cite the appropriate source(s) using their corresponding numbers. "
    "Every answer should include at least one source citation. "
    "Only cite a source when you are explicitly referencing it. "
    "If none of the sources are helpful, you should indicate that. "
    "For example:\n"
    "Source 1:\n"
    "The sky is red in the evening and blue in the morning.\n"
    "Source 2:\n"
    "Water is wet when the sky is red.\n"
    "Query: When is water wet?\n"
    "Answer: Water will be wet when the sky is red [2], "
    "which occurs in the evening [1].\n"
    "Now it's your turn. "
    "We have provided an existing answer: {existing_answer}"
    "Below are several numbered sources of information. "
    "Use them to refine the existing answer. "
    "If the provided sources are not helpful, you will repeat the existing answer."
    "\nBegin refining!"
    "\n------\n"
    "{context_msg}"
    "\n------\n"
    "Query: {query_str}\n"
    "Answer: "
)

DEFAULT_CITATION_CHUNK_SIZE = 512
DEFAULT_CITATION_CHUNK_OVERLAP = 20

In [7]:
from llama_index.core.schema import (
    MetadataMode,
    NodeWithScore,
    TextNode,
)

from llama_index.core.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

from typing import Union, List
from llama_index.core.node_parser import SentenceSplitter


class CitationQueryEngineWorkflow(Workflow):
    @step(pass_context=True)
    async def retrieve(
        self, ctx: Context, ev: StartEvent
    ) -> RetrieverEvent:
        "Entry point for RAG, triggered by a StartEvent with `query`."
        query = ev.get("query")
        if not query:
            return None

        print(f"Query the database with: {query}")

        # store the query in the global context
        ctx.data["query"] = query

        if ev.index is None:
            print("Index is empty, load some documents before querying!")
            return None
        else:
            print("Index is not empty, proceed with querying!")

        retriever = ev.index.as_retriever(similarity_top_k=2)
        nodes = retriever.retrieve(query)
        print(f"Retrieved {len(nodes)} nodes.")
        return RetrieverEvent(nodes=nodes)
    
    @step(pass_context=True)
    async def create_citation_nodes(
        self, ctx: Context, ev: RetrieverEvent
    ) -> CreateCitationsEvent:
        """
        Modify retrieved nodes to create granular sources for citations.

        Takes a list of NodeWithScore objects and splits their content
        into smaller chunks, creating new NodeWithScore objects for each chunk.
        Each new node is labeled as a numbered source, allowing for more precise
        citation in query results.

        Args:
            nodes (List[NodeWithScore]): A list of NodeWithScore objects to be processed.

        Returns:
            List[NodeWithScore]: A new list of NodeWithScore objects, where each object
            represents a smaller chunk of the original nodes, labeled as a source.
        """
        nodes = ev.nodes

        new_nodes: List[NodeWithScore] = []

        text_splitter = SentenceSplitter(
            chunk_size=DEFAULT_CITATION_CHUNK_SIZE,
            chunk_overlap=DEFAULT_CITATION_CHUNK_OVERLAP,
        )

        for node in nodes:
            text_chunks = text_splitter.split_text(
                node.node.get_content(metadata_mode=MetadataMode.NONE)
            )

            for text_chunk in text_chunks:
                text = f"Source {len(new_nodes)+1}:\n{text_chunk}\n"

                new_node = NodeWithScore(
                    node=TextNode.parse_obj(node.node), score=node.score
                )
                new_node.node.text = text
                new_nodes.append(new_node)
        return CreateCitationsEvent(nodes=new_nodes)

    @step(pass_context=True)
    async def synthesize(
        self, ctx: Context, ev: CreateCitationsEvent
    ) -> StopEvent:
        """Return a streaming response using the retrieved nodes."""
        llm = OpenAI(model="gpt-4")
        query = ctx.data.get("query")
        print(f"Synthesizing response for query: {query}")

        synthesizer = get_response_synthesizer(
            llm=llm,
            text_qa_template=CITATION_QA_TEMPLATE,
            refine_template=CITATION_REFINE_TEMPLATE,
            response_mode=ResponseMode.COMPACT,
            use_async=True,
        )

        response = await synthesizer.asynthesize(query, nodes=ev.nodes)
        print(f"Response: {response}")
        return StopEvent(result=response)

In [8]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(CitationQueryEngineWorkflow,filename="CitationQueryEngineWorkflow.html")

CitationQueryEngineWorkflow.html


In [9]:
w = CitationQueryEngineWorkflow()

In [10]:
result = await w.run(query="What is streamlit?", index=index)

Query the database with: What is streamlit?
Index is not empty, proceed with querying!
Retrieved 2 nodes.
Synthesizing response for query: What is streamlit?
Response: The provided sources do not contain information on what Streamlit is.


In [11]:
from IPython.display import Markdown, display

display(Markdown(f"{result}"))

The provided sources do not contain information on what Streamlit is.

In [12]:
print(result.source_nodes[1].node.get_text())

Source 2:
The Canada Student Grants  also saw a 40% increase in funding.[11]
Freeland issued $15B of spending  cuts, achieved by defunding public services and cancelling
previously announced programs.[12] A new tax 2% on stock buybacks  was also introduced.[11]Background
Measures
Housing6/19/24, 1:59 PM 2023 Canadian federal budget - Wikipedia
https://en.wikipedia.org/wiki/2023_Canadian_federal_budget 1/4



In [13]:
retriever = index.as_retriever()
retrieved_nodes = retriever.retrieve("What is llama_index?")

In [14]:
print(retrieved_nodes)

[NodeWithScore(node=TextNode(id_='efe653e2-8822-4209-a2d5-ac47fd7da608', embedding=None, metadata={'page_label': '2', 'file_name': '2023_canadian_budget.pdf', 'file_path': '/Users/boringtao/Projects/AutoRAG/notebooks/data/rag/2023_canadian_budget.pdf', 'file_type': 'application/pdf', 'file_size': 376126, 'creation_date': '2024-08-24', 'last_modified_date': '2024-08-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='98753a12-c109-4b52-aa66-09bc49886403', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '2', 'file_name': '2023_canadian_budget.pdf', 'file_path': '/Users/boringtao/Projects/AutoRAG/notebooks/data/rag/2023_canadian_budget.pdf', 'file_type': 'application/pdf', 'file_size': 376126, 

In [15]:
from llama_index.core.data_structs import Node
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.COMPACT
)

query_engine = index.as_query_engine(response_synthesizer=response_synthesizer)
response = query_engine.query("What is streamlit")

In [16]:
print(response)

Streamlit is a popular open-source Python library that is used to create web applications for machine learning and data science projects. It allows users to easily build interactive and customizable web interfaces for their Python scripts without requiring extensive web development knowledge.
